In [5]:
from tweepy import OAuthHandler
from tweepy import API
from tweepy import Cursor
from datetime import datetime, date, time, timedelta
from collections import Counter
import sys
import tweepy as tw
import numpy as np
import pandas as pd
import tweepy

class Import_tweet_sentiment:

    consumer_key="3Y48BcedbNpK0ShhnJgLfY6d1"
    consumer_secret="U3oUzM2vzqM4jHFVyffDYJdyAUrKzbRMdqiJZSXSc9goGO2ZoK"
    access_token="2555973745-pahK2Jnvazm2aHxRqO0HCJB2F6uSgaaXqWJM9UQ"
    access_token_secret="eCq2G42fVjtxAL3IxZxKZ8VQP8Ghk6sQ2sTYGHTOY4dyN"

    def tweet_to_data_frame(self, tweets):
        df = pd.DataFrame(data=[tweet.text for tweet in tweets], columns=['Tweets'])
        return df

    def get_tweets(self, handle):
        auth = OAuthHandler(self.consumer_key, self.consumer_secret)
        auth.set_access_token(self.access_token, self.access_token_secret)
        auth_api = API(auth)

        account = handle
        item = auth_api.user_timeline(id=account,count=50)
        df = self.tweet_to_data_frame(item)

        all_tweets = []
        for j in range(50):
            all_tweets.append(df.loc[j]['Tweets'])
        return all_tweets

    def get_hashtag(self, hashtag):
        auth = OAuthHandler(self.consumer_key, self.consumer_secret)
        auth.set_access_token(self.access_token, self.access_token_secret)
        auth_api = API(auth)
        account = hashtag
        all_tweets = []

        for tweet in tweepy.Cursor(auth_api.search, q=account, lang='en',tweet_mode='extended').items(50):
            
            status=tweet
            if 'extended_tweet' in status._json: 
                
                status_json = status._json['extended_tweet']['full_text'] 
                #status=tweet
                
            elif 'retweeted_status' in status._json and 'extended_tweet' in status._json['retweeted_status']: 
                status_json = status._json['retweeted_status']['extended_tweet']['full_text'] 
            elif 'retweeted_status' in status._json: 
                status_json = status._json['retweeted_status']['full_text'] 
            else: 
                status_json = status._json['full_text'] 
            print(status_json)
            all_tweets.append(status_json)
        return all_tweets

In [6]:
tweet_obj=Import_tweet_sentiment()

In [8]:
t=tweet_obj.get_tweets("MoRD_GOI")

In [11]:
for i in t:
    print(i)
    print(" ")

कोरोना महामारी की रोकथाम के लिए जागरूकता का बिगुल बजा रही हैं शहडोल की ग्रामीण SHG महिलाएँ। 

पढ़ें:… https://t.co/Je40UELlNe
 
#Unite2FightCorona https://t.co/IpgLCvxdi2
 
आप अपने बच्चों के रोल मॉडल होते हैं।इसलिए ज़रूरी है कि बाहर निकलने के पहले, आप हमेशा मास्क पहने। ऐसा करने से वो सुर… https://t.co/Sn4c7Ncu1o
 
छत्तीसगढ़ के जांजगीर में कोरोना की वजह से नहीं होगी रोजगार से दूरी,
सभी सावधानियों के साथ मनरेगा करेगा बाहर से लौटे… https://t.co/eOoHepL517
 
...ताकि, एक भी ज़िंदगी ना जाए हार।
हाथरस की ग्रमीण SHG महिलाएं कोरोना काल में मास्क बनाकर सहेज रहीं ज़िंदगी। 

पढें… https://t.co/ty0Usffozl
 
आइए, कल्याणी दीदी की सलाह के साथ कोरोना का मुकाबला करें और मास्क पहनना, साबुन से हाथ धोना तथा शारीरिक दूरी बनाए रख… https://t.co/jiFVifFifO
 
Mukhiya Ji brings to you the ideal way of protecting oneself from #COVID19

#Unite2FightCorona https://t.co/2aeUxUmCTa
 
On April 26, Nagaland inaugurated its first Construction Training Centre in Dimapur. It offers courses sponsored by… https://t.co/8aO1y

In [1]:


import urllib
import requests
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd
import os


def find_jobs_from(website, job_title, location, desired_characs, filename="results.xls"):    
    """
    This function extracts all the desired characteristics of all new job postings
    of the title and location specified and returns them in single file.
    The arguments it takes are:
        - Website: to specify which website to search (options: 'Indeed' or 'CWjobs')
        - Job_title
        - Location
        - Desired_characs: this is a list of the job characteristics of interest,
            from titles, companies, links and date_listed.
        - Filename: to specify the filename and format of the output.
            Default is .xls file called 'results.xls'
    """
    
    if website == 'Indeed':
        job_soup = load_indeed_jobs_div(job_title, location)
        jobs_list, num_listings = extract_job_information_indeed(job_soup, desired_characs)
    
    if website == 'CWjobs':
        location_of_driver = os.getcwd()
        driver = initiate_driver(location_of_driver, browser='chrome')
        job_soup = make_job_search(job_title, location, driver)
        jobs_list, num_listings = extract_job_information_cwjobs(job_soup, desired_characs)
    
    save_jobs_to_excel(jobs_list, filename)
 
    print('{} new job postings retrieved from {}. Stored in {}.'.format(num_listings, 
                                                                          website, filename))
    return jobs_list, num_listings
    

## ======================= GENERIC FUNCTIONS ======================= ##

def save_jobs_to_excel(jobs_list, filename):
    jobs = pd.DataFrame(jobs_list)
    jobs.to_excel(filename)



## ================== FUNCTIONS FOR INDEED.CO.IN =================== ##

def load_indeed_jobs_div(job_title, location):
    getVars = {'q' : job_title, 'l' : location, 'fromage' : 'last', 'sort' : 'date'}
    url = ('https://www.indeed.co.in/jobs?' + urllib.parse.urlencode(getVars))
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    job_soup = soup.find(id="resultsCol")
    return job_soup

def extract_job_information_indeed(job_soup, desired_characs):
    job_elems = job_soup.find_all('div', class_='jobsearch-SerpJobCard')
     
    cols = []
    extracted_info = []
    
    
    if 'titles' in desired_characs:
        titles = []
        cols.append('titles')
        for job_elem in job_elems:
            titles.append(extract_job_title_indeed(job_elem))
        extracted_info.append(titles)                    
    
    if 'companies' in desired_characs:
        companies = []
        cols.append('companies')
        for job_elem in job_elems:
            companies.append(extract_company_indeed(job_elem))
        extracted_info.append(companies)
    
    if 'links' in desired_characs:
        links = []
        cols.append('links')
        for job_elem in job_elems:
            links.append(extract_link_indeed(job_elem))
        extracted_info.append(links)
    
    if 'date_listed' in desired_characs:
        dates = []
        cols.append('date_listed')
        for job_elem in job_elems:
            dates.append(extract_date_indeed(job_elem))
        extracted_info.append(dates)
    
    jobs_list = {}
    
    for j in range(len(cols)):
        jobs_list[cols[j]] = extracted_info[j]
    
    num_listings = len(extracted_info[0])
    
    return jobs_list, num_listings


def extract_job_title_indeed(job_elem):
    title_elem = job_elem.find('h2', class_='title')
    title = title_elem.text.strip()
    return title

def extract_company_indeed(job_elem):
    company_elem = job_elem.find('span', class_='company')
    company = company_elem.text.strip()
    return company

def extract_link_indeed(job_elem):
    link = job_elem.find('a')['href']
    link = 'www.Indeed.co.in/' + link
    return link

def extract_date_indeed(job_elem):
    date_elem = job_elem.find('span', class_='date')
    date = date_elem.text.strip()
    return date

In [2]:
desired_chars=['titles','companies','links','date_listed']
jobs_list, num_listings=find_jobs_from("Indeed","python","india",desired_chars)

15 new job postings retrieved from Indeed. Stored in results.xls.


In [3]:
jobs_list['titles']

['Network Engineer -AVP\nnew',
 'Model Implementation consultant - Data Science\nnew',
 'Support Engineer\nnew',
 'Software Sr Engineer\nnew',
 'Support Escalation Engineer\nnew',
 'QA Analyst 2-ProdDev\nnew',
 'Technical Lead - Firmware Testing\nnew',
 'HIL Test Engineer\nnew',
 'Intern\nnew',
 'External Job Post\nnew',
 'Student Intern - Software Testing\nnew',
 'Student Intern - Enterprise Computing\nnew',
 'SOFTWARE DEVELOPER (RUBY) : SDR\nnew',
 'Storage Analyst\nnew',
 'Software Engineer, QA\nnew']

In [7]:

def clean_job_titles(words):
    titles=[]
    for i in words:
        text=i.split()

        for x in text:
            if x=='new':
                text.remove(x)
                status=True
        text=" ".join(text)
        titles.append(text)
        #print(text)
        
    return titles
words=clean_job_titles(jobs_list['titles'])
words

['Network Engineer -AVP',
 'Model Implementation consultant - Data Science',
 'Support Engineer',
 'Software Sr Engineer',
 'Support Escalation Engineer',
 'QA Analyst 2-ProdDev',
 'Technical Lead - Firmware Testing',
 'HIL Test Engineer',
 'Intern',
 'External Job Post',
 'Student Intern - Software Testing',
 'Student Intern - Enterprise Computing',
 'SOFTWARE DEVELOPER (RUBY) : SDR',
 'Storage Analyst',
 'Software Engineer, QA']

In [18]:
res=zip(words,jobs_list['companies'],jobs_list['links'],jobs_list['date_listed'])
res=list(res)
res

[('Network Engineer -AVP',
  'Wells Fargo',
  'www.Indeed.co.in//cmp/Wells-Fargo',
  'Just posted'),
 ('Model Implementation consultant - Data Science',
  'Wells Fargo',
  'www.Indeed.co.in//pagead/clk?mo=r&ad=-6NYlbfkN0A0sLjryQUNkc81K2goHfqpo9JHml6Vo2yWT4XuRGLXtt0Plp8naQ2M7aL2M5qwnGam8t39SYXnBw86yVHfu138I_DVfwSd-V5IkThflGly5x8_3iEB1nXLb4gaSyjBejMOr2DLWYlZRQlIhdYqhxhGVF74k_o6yap30owQxbofnD_JdXSeNzDrLw7Kahqynas6Qk-Aj2J3KsfHDPklHOpLakhQny0IDiespvd3m0htXxOI0U44ktfjqiLY9P8dXMshmkVOZPsFpzTg4Vd7D4SdEuDvpsUYTXGEDAnhUZ5dwq2u1s6XNQWbkCURu1A2wvimNJZ0TiTfpAwVesiE35e82x64bI07NfNb0G_1sxXCh224Hi-xrcIiGTtyUhi2l3tj6i6K5Al0py90RoYJiEMrNXlYMo-xDC0aBItYme0Gp_iENoFZgbLylSNvdarXTMsALr1NHuU8NYuRx4DakWDlP7iEQSbzlV5XdqqjFcR-M5TKUImcgjloXJYNbO5H1HHSXh5EUgH2zHHAR6QmgEL_caxqjyhTrqjKEEf1HqNZQ4mPMcKcUnAgg5Pv72H2wIL5Kz3dLE_8Da_U-96qoUP2IDJUILqZCihI4ruBDw3D3to_d9OnG_9w&p=1&fvj=0&vjs=3',
  'Just posted'),
 ('Support Engineer',
  'Microsoft',
  'www.Indeed.co.in//rc/clk?jk=06cdfb64f164635c&fccid=734cb5a01ee60f80&vjs=3

In [34]:
for i in res:
    print("Job-Title : {}".format(i[0]) , "\nCompany : {}".format(i[1]),
          "\nLink To Apply : {}".format(i[2]),"\nDate Listed : {}".format(i[3]))
    print(" ")

Job-Title : Network Engineer -AVP 
Company : Wells Fargo 
Link To Apply : www.Indeed.co.in//cmp/Wells-Fargo 
Date Listed : Just posted
 
Job-Title : Model Implementation consultant - Data Science 
Company : Wells Fargo 
Link To Apply : www.Indeed.co.in//pagead/clk?mo=r&ad=-6NYlbfkN0A0sLjryQUNkc81K2goHfqpo9JHml6Vo2yWT4XuRGLXtt0Plp8naQ2M7aL2M5qwnGam8t39SYXnBw86yVHfu138I_DVfwSd-V5IkThflGly5x8_3iEB1nXLb4gaSyjBejMOr2DLWYlZRQlIhdYqhxhGVF74k_o6yap30owQxbofnD_JdXSeNzDrLw7Kahqynas6Qk-Aj2J3KsfHDPklHOpLakhQny0IDiespvd3m0htXxOI0U44ktfjqiLY9P8dXMshmkVOZPsFpzTg4Vd7D4SdEuDvpsUYTXGEDAnhUZ5dwq2u1s6XNQWbkCURu1A2wvimNJZ0TiTfpAwVesiE35e82x64bI07NfNb0G_1sxXCh224Hi-xrcIiGTtyUhi2l3tj6i6K5Al0py90RoYJiEMrNXlYMo-xDC0aBItYme0Gp_iENoFZgbLylSNvdarXTMsALr1NHuU8NYuRx4DakWDlP7iEQSbzlV5XdqqjFcR-M5TKUImcgjloXJYNbO5H1HHSXh5EUgH2zHHAR6QmgEL_caxqjyhTrqjKEEf1HqNZQ4mPMcKcUnAgg5Pv72H2wIL5Kz3dLE_8Da_U-96qoUP2IDJUILqZCihI4ruBDw3D3to_d9OnG_9w&p=1&fvj=0&vjs=3 
Date Listed : Just posted
 
Job-Title : Support Engineer 
Company : M

In [2]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from numpy import array
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix
import datetime
import calendar

In [3]:
def get_train():
    data = pd.read_csv("train.csv",usecols = ['Avg Month Temp'])
    df=data.values
    X=df
    data = pd.read_csv("train.csv",usecols = ['Ratio'])
    df=data.values
    y=df
    # print("Y NEW SHAPE :",y.shape)
    X = np.reshape(X, (X.shape[0], 1, X.shape[1]))
    y = np.reshape(y, (y.shape[0]))
    # print("Y NEW CHANGED SHAPE :",y.shape)
    # print("X NEW :",X)
    # print("Y NEW :",y)
    return X, y
def get_test():
    data = pd.read_csv("test.csv",usecols = ['Avg Month Temp'])
    df=data.values
    X=df
    data = pd.read_csv("test.csv",usecols = ['Ratio'])
    df=data.values
    y=df
    # print("Y NEW SHAPE :",y.shape)
    X = np.reshape(X, (X.shape[0], 1, X.shape[1]))
    y = np.reshape(y, (y.shape[0]))
    # print("Y NEW CHANGED SHAPE :",y.shape)
    # print("X NEW :",X)
    # print("Y NEW :",y)
    return X, y





In [4]:
now = datetime.datetime.now()
year=now.year
month=now.month
day=now.day
one=year%10
print(one)
year=year/10
two=int(year%10)
print(two)
yr=two*10+one
print("Day ",day,"Month ",month,"Year ",yr)
monthname=calendar.month_name[(month+1)%12]
print("https://www.accuweather.com/en/in/ratnagiri/189289/"+str(monthname.lower())+"-weather/189289?monyr="+str((month+1)%12)+"/"+str(day)+"/"+str(yr)+"&view=table")
model = load_model('lstm_model.h5')


1
2
Day  29 Month  4 Year  21
https://www.accuweather.com/en/in/ratnagiri/189289/may-weather/189289?monyr=5/29/21&view=table


In [10]:
def get_estimate_yield(temp,area):
    X,ytrain=get_train()
    X,ytest = get_test()
    weather=temp
    area=area
    
    X=np.array([[[weather]]],np.float32)
    ytest = model.predict(X, verbose=0)

    yvalue=ytest[0]
    ratio=yvalue[0]
    
    possible_prod=area*ratio
    
    return round(possible_prod,2)

    
    

In [11]:
get_estimate_yield(30,200)

1695.98

In [1]:
import pandas as pd
import numpy as np


In [2]:
data_sever='Symptom_severity.csv'
data_precaution='symptom_precaution.csv'
data_desc='symptom_Description.csv'

In [14]:
df_prec=pd.read_csv(data_precaution)
df_prec

,Drug Reaction,stop irritation,consult nearest hospital,stop taking drug,follow up
0,Malaria,Consult nearest hospital,avoid oily food,avoid non veg food,keep mosquitos out
1,Allergy,apply calamine,cover area with bandage,NaN,use ice to compress itching
2,Hypothyroidism,reduce stress,exercise,eat healthy,get proper sleep
3,Psoriasis,wash hands with warm soapy water,stop bleeding using pressure,consult doctor,salt baths
4,GERD,avoid fatty spicy food,avoid lying down after eating,maintain healthy weight,exercise
5,Chronic cholestasis,cold baths,anti itch medicine,consult doctor,eat healthy
6,hepatitis A,Consult nearest hospital,wash hands through,avoid fatty spicy food,medication
7,Osteoarthristis,acetaminophen,consult nearest hospital,follow up,salt baths
8,(vertigo) Paroymsal Positional Vertigo,lie down,avoid sudden change in body,avoid abrupt head movment,relax
9,Hypoglycemia,lie down on side,check in pulse,drink sugary drinks,consult doctor


In [15]:
df_prec.columns=['Disease','Precaution1','Precaution2','Precaution3','Precaution4']
df_prec

,Disease,Precaution1,Precaution2,Precaution3,Precaution4
0,Malaria,Consult nearest hospital,avoid oily food,avoid non veg food,keep mosquitos out
1,Allergy,apply calamine,cover area with bandage,NaN,use ice to compress itching
2,Hypothyroidism,reduce stress,exercise,eat healthy,get proper sleep
3,Psoriasis,wash hands with warm soapy water,stop bleeding using pressure,consult doctor,salt baths
4,GERD,avoid fatty spicy food,avoid lying down after eating,maintain healthy weight,exercise
5,Chronic cholestasis,cold baths,anti itch medicine,consult doctor,eat healthy
6,hepatitis A,Consult nearest hospital,wash hands through,avoid fatty spicy food,medication
7,Osteoarthristis,acetaminophen,consult nearest hospital,follow up,salt baths
8,(vertigo) Paroymsal Positional Vertigo,lie down,avoid sudden change in body,avoid abrupt head movment,relax
9,Hypoglycemia,lie down on side,check in pulse,drink sugary drinks,consult doctor


,Disease,Precaution1,Precaution2,Precaution3,Precaution4
31,Jaundice,drink plenty of water,consume milk thistle,eat fruits and high fiberous food,medication


In [19]:
d='Jaundice'

val=df_prec[df_prec['Disease']==d]
val['Precaution1']


31    drink plenty of water
Name: Precaution1, dtype: object

In [24]:
def get_precautions(disease):
    data=df_prec[df_prec['Disease']==disease]
    prec1=data['Precaution1'].values[0]
    prec2=data['Precaution2'].values[0]
    prec3=data['Precaution3'].values[0]
    prec4=data['Precaution4'].values[0]
    
    return prec1,prec2,prec3,prec4
    

In [25]:
get_precautions('Typhoid')

('eat high calorie vegitables',
 'antiboitic therapy',
 'consult doctor',
 'medication')

In [26]:
get_precautions('Chronic cholestasis')

('cold baths', 'anti itch medicine', 'consult doctor', 'eat healthy')

In [29]:
descr=pd.read_csv(data_desc)
descr.columns=['Disease','Description']
descr

,Disease,Description
0,Malaria,An infectious disease caused by protozoan para...
1,Allergy,An allergy is an immune system response to a f...
2,Hypothyroidism,"Hypothyroidism, also called underactive thyroi..."
3,Psoriasis,Psoriasis is a common skin disorder that forms...
4,GERD,"Gastroesophageal reflux disease, or GERD, is a..."
5,Chronic cholestasis,"Chronic cholestatic diseases, whether occurrin..."
6,hepatitis A,Hepatitis A is a highly contagious liver infec...
7,Osteoarthristis,Osteoarthritis is the most common form of arth...
8,(vertigo) Paroymsal Positional Vertigo,Benign paroxysmal positional vertigo (BPPV) is...
9,Hypoglycemia,Hypoglycemia is a condition in which your blo...


In [36]:
def get_description(disease):
    data=descr[descr['Disease']==disease]
    
    des=data['Description'].values[0]   
    
    return des

In [37]:
get_description('Typhoid')

'An acute illness characterized by fever caused by infection with the bacterium Salmonella typhi. Typhoid fever has an insidious onset, with fever, headache, constipation, malaise, chills, and muscle pain. Diarrhea is uncommon, and vomiting is not usually severe.'